### Load tensorflow

In [1]:
import tensorflow as tf

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [2]:
!ls

sample_data


In [3]:
#You can use wget to download the file directly
!wget http://www.manythings.org/anki/hin-eng.zip --quiet

In [4]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [5]:
data[400:500]

'.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179123 (fastrizwaan)\nHello!\tनमस्'


### Extract Source and Target Language pairs

In [6]:
#Split by newline character
data =  data.split('\n')

#Show some Data
data[100:105]

['Bring him in.\tउसको अंदर ले आओ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #307895 (CK) & #475932 (minshirui)',
 'Come with us.\tहमारे साथ आओ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #433696 (CK) & #485546 (minshirui)',
 'Happy Easter!\tएसटर मुबारक हो!\tCC-BY 2.0 (France) Attribution: tatoeba.org #66762 (papabear) & #3189572 (pranjal710)',
 'Has Tom left?\tटॉम चला गया क्या?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2244563 (CK) & #3216282 (anubhav93)',
 'I am at home.\tमैं घर पर हूँ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #29049 (CK) & #505256 (minshirui)']

In [7]:
len(data)

2916

### Separate Source and Target pairs

In [8]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt,_ = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

### Separate Source and Target pairs..

In [9]:
encoder_text[100:105]

['Bring him in.',
 'Come with us.',
 'Happy Easter!',
 'Has Tom left?',
 'I am at home.']

In [10]:
decoder_text[100:105]

['<start> उसको अंदर ले आओ। <end>',
 '<start> हमारे साथ आओ। <end>',
 '<start> एसटर मुबारक हो! <end>',
 '<start> टॉम चला गया क्या? <end>',
 '<start> मैं घर पर हूँ। <end>']

### Tokenize Source language sentences

In [11]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer()
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers 
encoder_seq[100:105] #Display some converted sentences

[[400, 29, 9], [49, 34, 80], [155, 1310], [42, 35, 171], [2, 67, 28, 98]]

In [12]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Maximum sentence length for Source language:  22
Source language vocablury size:  2422


### Tokenize Target language sentences

In [13]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [14]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  3047


### Compare different sentences length

In [15]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

Length for sentence number 100:  3
Length for sentence number 2000:  7


In [16]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

Length for sentence number 100:  6
Length for sentence number 2000:  8


### How do we make it same?

### Padding the sentences

In [17]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')

In [18]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2915, 22)
Target data shape:  (2915, 27)


#### Integer to Word converter for Decoder data

In [19]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [20]:
int_to_word_decoder[15]

'हैं।'

### Building Decoder Output

In [21]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

#### Convert target data in one hot vector

In [22]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [23]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)    

In [24]:
decoder_target_one_hot.shape

(2915, 27, 3048)

### Building the Training Model

In [25]:
tf.keras.backend.clear_session()

In [26]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

#### Build Encoder

In [27]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(22,))

#Embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, encoder_embedding_size)

#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#---Following code has been commented out for Attention-------
#LSTM Layer and its output
#x, state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)

#Build a list to feed Decoder
#encoder_states = [state_h, state_c]

#### Build Encoder - Get all hidden states

In [28]:
#Create LSTM Layer and get All hidden states, last hidden and cell state
encoder_lstm = tf.keras.layers.LSTM(rnn_units,return_state=True, return_sequences=True)

#Get 3 outputs of LSTM Layer
encoder_all_h_states, state_h, state_c = encoder_lstm(encoder_embedding_output)

#Build a list to feed Decoder
encoder_states = [state_h, state_c]

In [29]:
encoder_all_h_states

<KerasTensor: shape=(None, 22, 256) dtype=float32 (created by layer 'lstm')>

#### Build Decoder

In [31]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(27,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
decoder_all_h_states,_,_ = decoder_rnn(decoder_embedding_output, 
                                       initial_state=encoder_states)

#---Following code has been commented out for Attention-------
#Output Layer
#decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#Output of Dense layer
#decoder_outputs = decoder_dense(x)

In [32]:
decoder_all_h_states

<KerasTensor: shape=(None, 27, 256) dtype=float32 (created by layer 'lstm_1')>

#### Build Decoder...Alignment Matrix

In [33]:
#1. Dot Product between Decoder_all_h_states and encoder_all_h_states
#2. Apply softmax to get Alignment matrix

#Dimensions details
#decoder_all_states = batch_size x max_decoder_length x rnn_units
#encoder_all_states = batch_size x max_encoder_length x rnn_units
#score = batch_size x max_decoder_length x max_encoder_length
#alignment matrix = batch_size x max_decoder_length x max_encoder_length

score = tf.keras.layers.dot([decoder_all_h_states, encoder_all_h_states], axes=2)
alignment_matrix = tf.keras.layers.Activation('softmax')(score)

#Try general and concat approaches to alignment matrix

In [34]:
alignment_matrix

<KerasTensor: shape=(None, 27, 22) dtype=float32 (created by layer 'activation')>

#### Build Decoder...Context Vector

In [35]:
#Weighted sum of multiplication of Alignment matrix and encoder states
#Dimension of context_vector =  batch_size x max_decoder_length x rnn_units

context_vector = tf.keras.layers.dot([alignment_matrix, encoder_all_h_states], axes=[2,1])

In [36]:
context_vector

<KerasTensor: shape=(None, 27, 256) dtype=float32 (created by layer 'dot_1')>

#### Build Decoder...Attention Vector

In [37]:
#Concatenate context vector and decoder_all_h_states
#context_decoder_hidden = batch_size x max_decoder_length x rnn_units
#attention_vector = batch_size x max_decoder_length x 128

context_decoder_hidden = tf.keras.layers.concatenate([context_vector, 
                                                      decoder_all_h_states])

attention_dense_layer = tf.keras.layers.Dense(128, use_bias=False, 
                                              activation='relu')

attention_vector = attention_dense_layer(context_decoder_hidden)

In [38]:
attention_vector

<KerasTensor: shape=(None, 27, 128) dtype=float32 (created by layer 'dense')>

#### Build Decoder...Output layer

In [40]:
#Output layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#With attention input will be attention_vector and not decoder_all_h_states
decoder_outputs = decoder_dense(attention_vector)

In [41]:
decoder_outputs

<KerasTensor: shape=(None, 27, 3048) dtype=float32 (created by layer 'dense_1')>

### Build Model using both Encoder and Decoder

In [42]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [43]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [44]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 22)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 27)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 22, 50)       121150      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 27, 50)       152400      input_2[0][0]                    
______________________________________________________________________________________________

### Train the model

In [45]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=25,
          validation_split=0.2)

Epoch 1/25
37/37 [==============================] - 19s 420ms/step - loss: 5.5297 - accuracy: 0.6568 - val_loss: 3.0604 - val_accuracy: 0.5928
Epoch 2/25
37/37 [==============================] - 14s 391ms/step - loss: 1.6720 - accuracy: 0.7411 - val_loss: 3.1239 - val_accuracy: 0.5958
Epoch 3/25
37/37 [==============================] - 14s 391ms/step - loss: 1.5774 - accuracy: 0.7467 - val_loss: 3.0422 - val_accuracy: 0.5959
Epoch 4/25
37/37 [==============================] - 14s 390ms/step - loss: 1.5132 - accuracy: 0.7510 - val_loss: 2.8482 - val_accuracy: 0.6055
Epoch 5/25
37/37 [==============================] - 14s 388ms/step - loss: 1.4183 - accuracy: 0.7704 - val_loss: 2.8108 - val_accuracy: 0.6228
Epoch 6/25
37/37 [==============================] - 14s 390ms/step - loss: 1.3822 - accuracy: 0.7789 - val_loss: 2.7486 - val_accuracy: 0.6333
Epoch 7/25
37/37 [==============================] - 14s 393ms/step - loss: 1.3481 - accuracy: 0.7820 - val_loss: 2.8157 - val_accuracy: 0.6330

### Save the model for later reuse

In [ ]:
#model.save('models/seq2seq_training_translation_attention.hd5')

In [ ]:
model = tf.keras.models.load_model('models/seq2seq_training_translation_attention.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [ ]:
encoder_model = tf.keras.models.Model(inputs=encoder_inputs, #Padded input sequences
                                      outputs=[encoder_all_h_states] + #Hidden states at all time steps
                                      encoder_states) #Hidden state and Cell state at last time step

### Build the Decoder Model 
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
    <li><font color="blue">Define Input for all encoder states - Attention Layer </font></li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li><font color="blue">Build Attention Layer</font></li>
<li><font color="blue">Get Decoder Dense layer output using Attention vector</font></li>
    <li><font color="blue">Build Model</font></li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [ ]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Define Input encoder states - Attention Layer

In [ ]:
encoder_outputs = tf.keras.layers.Input(shape=(max_encoder_seq_length, rnn_units,))

##### Step 3 - Get Decoder RNN outputs along with h and c state

In [ ]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 4 - Build Attention Layer

In [ ]:
#Alignment score
p_score = tf.keras.layers.dot([rnn_outputs, encoder_outputs], axes=2)

#Perform softmax to get Alignment matrix
p_alignment_matrix = tf.keras.layers.Activation('softmax')(p_score)

#Context Vector
p_context_vector = tf.keras.layers.dot([p_alignment_matrix, encoder_outputs], axes=[2,1])

#Build Attention Vector
# 1. Caoncatenate both context vector and decoder outputs
# 2. Feed it to the Dense layer 
p_context_decoder_hidden = tf.keras.layers.concatenate([p_context_vector, rnn_outputs])
p_attention_vector = attention_dense_layer(p_context_decoder_hidden)

In [ ]:
p_alignment_matrix

<KerasTensor: shape=(None, 27, 22) dtype=float32 (created by layer 'activation_1')>

##### Step 5 - Get Decoder Dense layer output

In [ ]:
decoder_outputs = decoder_dense(p_attention_vector)

##### Step 6 - Build Decoder Model

In [ ]:
#3 Inputs - Word, h/c state and all hidden states from encoder
#3 Outputs - predicted word, h and c state values for next run and alignment matrix for visualization

decoder_model = tf.keras.models.Model([decoder_inputs] +  #Start sequence and then word
                                      decoder_states_inputs + #h and c state value for initialization
                                      [encoder_outputs],  #Encoder all hidden states for Attention layer
                                      [decoder_outputs] + #Model word prediction
                                      decoder_states +   #h and c states for next run
                                      [p_alignment_matrix]) #for Alignment matrix

# Predicting output from Seq2Seq model

##### Build a prediction function

In [ ]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values
    encoder_output =  encoder_model.predict(input_sequence)
    decoder_initial_states_value = encoder_output[1:]    
    encoded_seqs = encoder_output[0]
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        #Decoder model with 3 inputs
        predicted_outputs, h, c, a_matrix = decoder_model.predict([target_seq] + 
                                                                  decoder_initial_states_value +
                                                                  [encoded_seqs])
        
        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter index
        if (predicted_output == 0):
            predicted_word = ' '
        else:
            predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
        print(a_matrix[0][0])
    
    return predicted_sentence

##### Call Prediction function on a random sentence

In [ ]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 1):
    input_seq = encoder_input_data[i : i+1]
    #print(input_seq)
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.7257522e-38
 2.2333274e-28 2.8964719e-23 5.8233591e-22 7.9554605e-22 7.4699857e-22
 7.1089560e-22 6.9270803e-22 6.8192176e-22 6.7467676e-22 6.6960518e-22
 6.6597258e-22 6.6334574e-22 3.5199642e-01 5.7034897e-17 1.4445786e-17
 6.4800358e-01 1.4156054e-18]
[1.5960951e-28 1.2753956e-27 4.8009357e-26 2.8767649e-23 5.8000302e-19
 9.8376041e-15 9.1687194e-13 2.2439971e-12 2.1698270e-12 1.9695775e-12
 1.8592181e-12 1.7977728e-12 1.7592386e-12 1.7332331e-12 1.7150411e-12
 1.7020062e-12 1.6925209e-12 5.7460177e-01 9.7773886e-11 1.2535034e-10
 4.2539823e-01 1.5655536e-11]
[4.8648759e-27 3.2611947e-26 9.2650846e-25 3.5856389e-22 4.3836999e-18
 6.5109844e-14 7.9814809e-12 2.4909907e-11 2.6364862e-11 2.4354879e-11
 2.2978829e-11 2.2164713e-11 2.1651351e-11 2.1308524e-11 2.1071362e-11
 2.0903475e-11 2.0782143e-11 5.5874556e-01 1.0361688e-09 1.1667703e-09
 4.4125444e-01 2.4032337e-10]
[1.3910695e-26 9.5472649e-26 2.8698282e-24 1.3503641e-21 3

##### Save encoder and decoder model

In [ ]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('models/seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('models/seq2seq_decoder_eng_hin.hd5')  #Decoder model

##### Save encoder and decoder tokenizers

In [ ]:
import pickle

pickle.dump(encoder_t,open('models/encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('models/decoder_tokenizer_hin','wb'))